In [2]:

'''
    MNIST digit classification using tensorflow
    dataset: Digit recognizer (Kaggle)
'''

import tensorflow as tf
import pandas as pd

In [3]:
# importing the training set
data = pd.DataFrame(pd.read_csv('train.csv'))
data.sample(frac=1)
data = data.as_matrix()

In [50]:
import numpy as np

# converting the labels to hot matrix
def hot_matrix(digit_label, size):
    hm = np.zeros([size, 10], dtype = np.float)
    
    for j in range(size):
        hm[j, digit_label[j]] = 1
    return hm

# create batches
def create_batch(X, Y, size):
    # total batches
    
    idx = np.random.randint(np.size(Y[:, 0]), size = 100)
    idx = idx.reshape(size, 1)
    x_bat = X[idx]
    x_bat = x_bat.reshape(size, 784)
    y_bat = Y[idx]
    y_bat = y_bat.reshape(size, 10)
    return x_bat, y_bat

In [30]:
# data info

# training pixel values
pixel_values = data[0:38000, 1:]

# dataset length
m = np.size(pixel_values[:, 0])
digits = data[0:38000, 0:1]
# labels
labels = hot_matrix(digits, m)

print('pixel_values shape: ', pixel_values.shape)
print('labels shape: ', labels.shape)

pixel_values shape:  (38000, 784)
labels shape:  (38000, 10)


In [87]:
# placeholder for pixel_values
x = tf.placeholder(tf.float32, [None, 784])
# placeholder for labels
y = tf.placeholder(tf.float32, [None, 10])


# total layers = 3
# weights
w1 = tf.Variable(np.random.randn(784, 30), dtype = tf.float32)
w2 = tf.Variable(np.random.randn(30, 30), dtype = tf.float32)
w3 = tf.Variable(np.random.randn(30, 10), dtype = tf.float32)

# biases
b1 = tf.Variable(np.random.randn(30), dtype = tf.float32)
b2 = tf.Variable(np.random.randn(30), dtype = tf.float32)
b3 = tf.Variable(np.random.randn(10), dtype = tf.float32)

In [113]:
# hidden layers
z1 = tf.matmul(x, w1) + b1
a1 = tf.nn.sigmoid(z1)
z2 = tf.matmul(a1, w2) + b2
a2 = tf.nn.sigmoid(z2)
z3 = tf.matmul(a2, w3) + b3

# optimization function softmax
sigmoid_cross_entropy = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = z3, labels = y))

# gradient descent
optimize = tf.train.AdamOptimizer(0.005).minimize(sigmoid_cross_entropy)

In [114]:
# learning starts now
epochs = 35000
# creating Session
s = tf.InteractiveSession()

# initializing variables
tf.global_variables_initializer().run()

for i in range(epochs):
    x_batch, y_batch = create_batch(pixel_values, labels, 100)
    train_step = s.run(optimize, {x: x_batch, y: y_batch})
cost = s.run(sigmoid_cross_entropy, {x: pixel_values, y: labels})
print('cost: ', cost)

cost:  0.054982


In [115]:
# test set accuracy
test_data = data[36000:]
# test set size
m_test = np.size(test_data[:, 0])
# test set pixel values
test_pixel_values = test_data[:, 1:]
# creating hot matrix of test set labels
test_labels =  hot_matrix(test_data[:, 0:1], m_test)

# calculating accuracy
final_train = tf.nn.sigmoid(z3)

# training set prediction
train_predition = s.run(final_train, {x: pixel_values, y:labels})

# test set prediction
test_predition = s.run(final_train, {x: test_pixel_values, y:test_labels})

# training set accuracy
correct_train = tf.equal(tf.argmax(train_predition, 1), tf.argmax(labels, 1))
training_accuracy = tf.reduce_mean(tf.cast(correct_train, dtype = 'float'))

# test set accuracy
correct_test = tf.equal(tf.argmax(test_predition, 1), tf.argmax(test_labels, 1))
test_accuracy = tf.reduce_mean(tf.cast(correct_test, dtype = 'float'))

print('training accuracy: ', s.run(train_accuracy))
print('test accuracy: ', s.run(test_accuracy))

training accuracy:  0.774737
test accuracy:  0.902
